In [1]:
# 設定超參數HyperParameters 
batch_size =  64
epochs = 10
image_size = 128
lr=0.01

## 一、載入相關套件

In [2]:
import os
import cv2
import csv
import random
import time
import numpy as np
import pandas as pd
import matplotlib.image as mpimg # mpimg 用於讀取圖片
import matplotlib.pyplot as plt # plt 用於顯示圖片
import seaborn as sns

In [3]:
import tensorflow as tf

from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import utils as np_utils
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, Dense, Activation, Conv2DTranspose

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
# Print Start time
from datetime import datetime, timezone
import pytz

tpe = pytz.timezone('Asia/Taipei')
utc_start = datetime.now()
print("Start Time", tpe.fromutc(utc_start))

Start Time 2020-05-30 00:58:51.966285+08:00


## 建立Model

In [ ]:
model_resnet = InceptionResNetV2(weights='imagenet', include_top=False)

input = Input( shape=(image_size,image_size,3),name = 'image_input' )
output_conv = model_resnet(input)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
219062272/219055592 [==============================] - 3s 0us/step


In [ ]:
## 由於該問題是分 3 類，需要在最後修改 Dense(3)，softmax 代表 output 是機率  
#Add the fully-connected layers 
p = Flatten()(output_conv)
p = Dense(256,activation = 'relu')(p)
p = Dropout(0.2)(p)
p = Dense(3, activation='softmax', name='predictions')(p)

model = Model(inputs=input, outputs=p)

In [ ]:
adam = optimizers.Adam(lr=lr)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['acc'])

## 製作標籤&資料集

In [ ]:
from imgGen import imgGenFunc

In [ ]:
# Read testing data
flip=0; rotate=0
x_test, y_test = imgGenFunc(image_size, flip, rotate, "dev.csv", "C1-P1_Dev/")

### Test: flip=1, rotate=1, blur=1, validation=original_image

In [ ]:
flip = 1; rotate = 1
x_train, y_train = imgGenFunc(image_size, flip, rotate, "train.csv", "C1-P1_Train/")
x_train_blur, y_train_blur = imgGenFunc("train.csv", "C1-P1_Train/")

x_train.append(x_train_blur)
y_train.append(y_train_blur)

# shuffle the data
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

In [ ]:
# Model檔名設定
file_name = str(epochs)+'_'+str(batch_size)

In [ ]:
# 加入EarlyStopping以及Tensorboard等回調函數
CB = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
TB = keras.callbacks.TensorBoard(log_dir='./log'+"_"+file_name, histogram_freq=1)

In [ ]:
datagen = ImageDataGenerator(
    zca_whitening=False,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

In [ ]:
datagen.fit(x_train)

In [ ]:
### Start Training
history_1 = model.fit(
    datagen.flow(x_train , y_train, batch_size=batch_size),
    epochs = epochs,
    validation_data = (x_test, y_test)
)

In [ ]:
# Print Finish Training time
tpe = pytz.timezone('Asia/Taipei')
utc_finish = datetime.now()
print("Finish Training Time", tpe.fromutc(utc_finish))

difference = utc_finish-utc_start
seconds_in_day = 24 * 60 * 60
divmod(difference.days * seconds_in_day + difference.seconds, 60)

## 繪製Model學習成效

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,2)
    plt.show()
    
plot_learning_curves(history_1)

## 測試集預測準確度

In [ ]:
# 測試集標籤預測
y_pred = model.predict(x_test)

In [ ]:
# 模型預測後的標籤
predict_label = np.argmax(y_pred,axis=1)
print(predict_label)

In [ ]:
# 模型原標籤
true_label = np.argmax(y_test,axis=1)
print(true_label)

In [ ]:
pd.crosstab(true_label,predict_label,rownames=['實際值'],colnames=['預測值'])

In [ ]:
# 整體準確度
count = 0
for i in range(len(y_pred)):
    if(np.argmax(y_pred[i]) == np.argmax(y_test[i])): #argmax函数找到最大值的索引，即为其类别
        count += 1
score = count/len(y_pred)
print('正确率为:%.2f%s' % (score*100,'%'))

In [ ]:
# 儲存模型相關參數
# model.save('h5/'+file_name+'.h5')